#Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#==================================================================
# to NST_net
#==================================================================
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.models import vgg19
import requests
from IPython.display import clear_output
%matplotlib inline

In [3]:
TOKEN = '1663109481:AAEAu9pghz8DNRcUdT64s5O4tGmaEMq0H0s'

#NST_net

In [4]:
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
backbone = vgg19(pretrained=True).features.to(device).eval().requires_grad_(False)

for (i, layer) in enumerate(backbone):
  if isinstance(layer, torch.nn.MaxPool2d):
    backbone[i] = torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


In [6]:
def load_image(name_img, max_size=256, shape=None):
    
    image = Image.open(name_img).convert('RGB')
    
    if max(image.size) > max_size:
        size = max_size
    else:
        size = max(image.size)
    
    if shape is not None:
        size = shape
        
    in_transform = transforms.Compose([
                        transforms.Resize(size),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])

    # discard the alpha channel (that's the :3) and add the batch dimension
    image = in_transform(image)[:3,:,:].unsqueeze(0)
    
    return image

In [7]:
# convert tesnor to image, denormalize
def im_convert(tensor):
  # tensor.detach() creates a tensor that shares storage with tensor that does not require grad. 
  # It detaches the output from the computational graph. So no gradient will be backpropagated along this variable.
  image = tensor.to("cpu").clone().detach()
  image = image.numpy().squeeze()
  image = image.transpose(1, 2, 0) # to -> H, W, C
  image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
  # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
  image = image.clip(0, 1)
  return image

##Net building and Training

In [8]:
def gram_matrix(tensor):
    batch_size, channels, height, width = tensor.size()
    
    tensor = tensor.view(batch_size * channels, height * width)
    gram = tensor @ tensor.t()
    
    return gram

def get_features(image, model, layers=None):
    if layers is None:
        layers = {'0': 'conv1_1',
                  '5': 'conv2_1', 
                  '10': 'conv3_1', 
                  '19': 'conv4_1',
                  '21': 'conv4_2',  # content layer
                  '28': 'conv5_1'}
    features = {}
    x = image
    
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
            
    return features

In [9]:
def NST_fit(content, style, backbone, num_epochs=3000):

  # get_content features
  content_features = get_features(content, backbone)
  # grad for content
  target = content.clone().requires_grad_(True).to(device)

  # get_style features
  style_features = get_features(style, backbone)
  # gram matrix for all in style_features
  style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

  # learn pams/optim
  optimizer = optim.AdamW([target], lr=3e-3)

  style_weights = {'conv1_1': 0.1,
                  'conv2_1': 0.2,
                  'conv3_1': 0.2,
                  'conv4_1': 0.5,
                  'conv5_1': 0.75}
  content_weight = 1 
  style_weight = 1e5
  
  # train loop
  for ii in range(1, num_epochs+1):

      # refresh target_features
      target_features = get_features(target, backbone)
      # refresh_content_loss
      content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2'])**2)
      # init style_loss
      style_loss = 0

      for layer in style_weights:
          # get the "target" style representation for the layer
          target_feature = target_features[layer]

          _, d, h, w = target_feature.shape
          
          # get the "target" gram matrix
          target_gram = gram_matrix(target_feature)
          # get the "style" style representation
          style_gram = style_grams[layer]

          # the style loss for one layer, weighted appropriately
          layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram)**2)
          # add to the style loss
          style_loss += layer_style_loss / (d * h * w)

      total_loss = content_weight * content_loss + style_weight * style_loss 
      
      # update your target image
      optimizer.zero_grad()
      total_loss.backward()
      optimizer.step()

  return target

#Telebot

In [10]:
!pip3 install PyTelegramBotAPI

     |████████████████████████████████| 81kB 5.9MB/s 
  Created wheel for PyTelegramBotAPI: filename=pyTelegramBotAPI-3.7.6-cp36-none-any.whl size=59261 sha256=2e87dd52a327ac4dfa98f00b088101a3d4ef0b096c3768a2366762ac71c61852
  Stored in directory: /root/.cache/pip/wheels/f6/66/6e/d42e8fcb446d2683b5afe0b23318f4bb58896bad26549c47b9
Successfully built PyTelegramBotAPI


In [11]:
import telebot
import os

In [15]:
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет, вы написали NST_bot(у)!\
                                      \nЯ могу перенести стиль одной картинки на другую.\
                                      \nЕсли Вам нужна помощь, то напишите --> /help')
@bot.message_handler(commands=['help'])
def help_message(message):
    bot.send_message(message.chat.id, 
                'В данном боте есть несколько методов переноса стиля:\
                \n(1) <ваш стиль на вашу картинку>.\
                \n Для этого Вы присылаете сразу две картинки: первая фото-контекст, вторая фото-стиль ПОСЛЕ пишите --> /My\
                \n(2) <стиль вангога/волны на вашу картинку>\
                \n Для этого Вы присылаете фото-контекст ПОСЛЕ пишите --> /VanGog или /Wave')

@bot.message_handler(content_types=['photo'])  
def get_images(message): 
  if message.media_group_id:

    file_info = bot.get_file(message.photo[len(message.photo) - 1].file_id)

    downloaded_file = bot.download_file(file_info.file_path)

    src = '/content/files/' + file_info.file_path
    with open(src, 'wb') as new_file:
        new_file.write(downloaded_file)
    
    bot.reply_to(message, "Принял")      

    path = '/content/files/photos' # Путь к вашей папке

    # Получим список имен всего содержимого папки
    # и превратим их в абсолютные пути
    dir_list = [os.path.join(path, x) for x in os.listdir(path)]

    if dir_list:
        # Создадим список из путей к файлам и дат их создания.
        date_list = [[x, os.path.getctime(x)] for x in dir_list]

        # Отсортируем список по дате создания в обратном порядке
        sort_date_list = sorted(date_list, key=lambda x: x[1], reverse=True)

        # Выведем первый элемент списка. Он и будет самым последним по дате
        print(str(sort_date_list[0][0]))
        print(str(sort_date_list[1][0]))

        style_im = Image.open(str(sort_date_list[0][0])).convert('RGB')
        style_im.save('/content/style.jpg')
        context_im = Image.open(str(sort_date_list[1][0])).convert('RGB')
        context_im.save('/content/context.jpg')

  else:
    file_info = bot.get_file(message.photo[-1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    with open('/content/context.jpg', "wb") as new_file:
      new_file.write(downloaded_file)
    bot.send_message(message.chat.id, 'Ваше фото-контекст получено!')

@bot.message_handler(commands=['VanGog'])
def handle_NST_VanGog(message):
  bot.send_message(message.chat.id, 'Стиль Ван Гога на ваше фото скоро будет перенесен')
  # content im.
  content = load_image('/content/context.jpg').to(device)
  # styles im., resize style to content
  style = load_image('/content/drive/MyDrive/NST_VanGog.jpg',
                        shape=content.shape[-2:]).to(device)
  target = NST_fit(content, style, backbone)
  im = Image.fromarray((im_convert(target) * 255).astype(np.uint8))
  bot.send_photo(message.chat.id, im)

@bot.message_handler(commands=['Wave'])
def handle_NST_Wave(message):
  bot.send_message(message.chat.id, 'Стиль Волны Хокусая на ваше фото скоро будет перенесен')
  # content im.
  content = load_image('/content/context.jpg').to(device)
  # styles im., resize style to content
  style = load_image('/content/drive/MyDrive/NST_Wave.jpg',
                        shape=content.shape[-2:]).to(device)
  target = NST_fit(content, style, backbone)
  im = Image.fromarray((im_convert(target) * 255).astype(np.uint8))
  bot.send_photo(message.chat.id, im)

@bot.message_handler(commands=['My'])
def handle_NST_My(message):
  bot.send_message(message.chat.id, 'Прошу Вас немного подождать')
  # content im.
  content = load_image('/content/context.jpg').to(device)
  # styles im., resize style to content
  style = load_image('/content/style.jpg',
                        shape=content.shape[-2:]).to(device)
  target = NST_fit(content, style, backbone)
  im = Image.fromarray((im_convert(target) * 255).astype(np.uint8))
  bot.send_photo(message.chat.id, im)

bot.polling()

/content/files/photos/file_26.jpg
/content/files/photos/file_25.jpg
/content/files/photos/file_26.jpg
/content/files/photos/file_25.jpg
